In [ ]:
import os
import xarray as xr
import numpy as np
import salvus.namespace as sn
import salvus.mesh as sm
import generator

from salvus.toolbox.helpers.wavefield_output import WavefieldOutput, wavefield_output_to_xarray
from salvus.flow.simple_config.boundary import Absorbing
from salvus.mesh.unstructured_mesh_utils import extract_model_to_regular_grid

SALVUS_FLOW_SITE_NAME = os.environ.get("SITE_NAME", "YOUR_SITE")
PROJECT_DIR = "YOUR_PROJECT_DIR"
save_path = "../data/"  # where to save the generated data

In [ ]:
# Simulation setting
center_frequency = 0.3
max_frequency_in_hertz = 0.6
start_time_in_seconds = -4
end_time_in_seconds = 40.0
time_step_in_seconds = 1e-3
sampling_interval_in_time_steps = 100 
elements_per_wavelength = 2  # /1
tensor_order = 4  # polynomial degree, must be one of 1, 2, 4
ranks_per_job = 1

# Absorbing boundary condition
reference_velocity = 1500
number_of_wavelengths = 3.5
abwidth = sm.absorbing_boundary.get_absorbing_boundary_width(velocity=reference_velocity, 
                                                             lowest_frequency=center_frequency, 
                                                             number_of_wavelengths=number_of_wavelengths)
extra = 1000
abwidth = abwidth + extra

wavelet = sn.simple_config.stf.Ricker(center_frequency=center_frequency)

abc = Absorbing(width_in_meters=abwidth-extra, 
                    side_sets=['x0','x1','y1'],
                    taper_amplitude=center_frequency,
                    side_sets_are_axis_aligned=True)

In [ ]:
# Full domain = computational domain + boundaries
xrange = 80700.414
yrange = 40000
dx = xrange / (256 - 1)
dy = yrange / (128 - 1)
xcomp = np.arange(0, xrange+dx, dx)
ycomp = np.arange(0, yrange+dy, dy)
xsim = np.arange(-abwidth, xrange+abwidth+dx, dx)
ysim = np.arange(0, yrange+abwidth+dy, dy)
nx, ny = len(xsim), len(ysim)
nxin, nyin = len(xcomp), len(ycomp)

In [ ]:
vsbg = np.load("../model/vsbg.npy")  # background model from CVM-S4.26

In [ ]:
iS = 0
nsim = 1
while iS < nsim:
    try:
        !rm -rf YOUR_PROJECT_DIR
        
        vssim = generator.generate_vs(nxin, nyin, vsbg)
        vpsim = generator.generate_vp(nxin, nyin, vssim)
        rhosim = generator.brocher_rho_from_vp(vpsim)
        
        vssim = generator.add_boundary_layers(vssim, nx, ny, nxin, nyin)
        vpsim = generator.add_boundary_layers(vpsim, nx, ny, nxin, nyin)
        rhosim = generator.add_boundary_layers(rhosim, nx, ny, nxin, nyin)
        
        # Xarray dataset
        ds = xr.Dataset(
            coords={"x": xsim, "y": ysim},
            data_vars={
                "VP": (["x", "y"], vpsim),
                "VS": (["x", "y"], vssim),
                "RHO": (["x", "y"], rhosim),
            },
        )

        # Generate source location
        srcxy = np.array([np.random.uniform(xcomp[0], xcomp[-1]+1e-5), 0])

        src = sn.simple_config.source.cartesian.VectorPoint2D(
                x=srcxy[0], y=srcxy[1], fx=0.0, fy=1.0
            )

        recs = sn.simple_config.receiver.cartesian.Point2D(
                x=srcxy[0],
                y=srcxy[1],
                station_code=f"{50:05d}",
                fields=["velocity"],
                )

        # Creating mesh
        model = sn.model.volume.cartesian.GenericModel(name="volume", data=ds)
        p = sn.Project.from_volume_model(PROJECT_DIR, model, True)

        sc = sn.SimulationConfiguration(
            name="volumetric_model",
            tensor_order=tensor_order,
            elements_per_wavelength=elements_per_wavelength,
            max_frequency_in_hertz=max_frequency_in_hertz,
            model_configuration=sn.ModelConfiguration(
                background_model=None,
                volume_models="volume",
            ),
            event_configuration=sn.EventConfiguration(
                wavelet=wavelet,
                waveform_simulation_configuration=sn.WaveformSimulationConfiguration(
                    start_time_in_seconds=start_time_in_seconds,
                    end_time_in_seconds=end_time_in_seconds,
                    time_step_in_seconds=time_step_in_seconds,
                    spectral_element_order=tensor_order,
                    boundary_conditions=abc,
                    ),
            ),
        )
            
        p.add_to_project(sc, overwrite=True)
        p.add_to_project(sn.Event(event_name="event_0", sources=src, receivers=recs))

        mesh = p.simulations.get_mesh("volumetric_model")

        p.simulations.launch(
            ranks_per_job=ranks_per_job,
            site_name=SALVUS_FLOW_SITE_NAME,
            events=p.events.list(),
            simulation_configuration="volumetric_model",
            extra_output_configuration={
                "volume_data": {
                    "sampling_interval_in_time_steps": sampling_interval_in_time_steps,
                    "fields": ["velocity"],
                    },
            },
        )

        p.simulations.query(block=True)

        vout_path = (
            p.simulations.get_simulation_output_directory(
                simulation_configuration="volumetric_model", event=p.events.list()[0]
            )
            / "volume_data_output.h5"
        )
        wavefield_output = WavefieldOutput.from_file(filename=vout_path, field='velocity', output_type='volume')

        # Extracting data to regular grid
        nx1, ny1 = nxin, nyin
        x1 = np.linspace(xcomp[0], xcomp[-1], nx1)
        y1 = np.linspace(ycomp[0], ycomp[-1], ny1)
        xx1, yy1 = np.meshgrid(x1, y1, indexing='ij')
        points = np.stack((xx1.flatten(), yy1.flatten()), axis=1)
        u = wavefield_output_to_xarray(wo=wavefield_output, points=points).values  # (nt, nc, nx, nz)
        u = u.reshape(u.shape[0], u.shape[1], nx1, ny1)
        u = u.transpose(2, 3, 0, 1)  # (nx, nz, nt, nc)

        # Starting from t = 0 or t = start_time_in_seconds
        #i0 = round(-start_time_in_seconds / (time_step_in_seconds * sampling_interval_in_time_steps))
        i0 = 0
        np.save(save_path+"u{}.npy".format(iS), u[:, :, i0:, :])

        ds_extract_2d = xr.Dataset(
            coords={
                "x": x1,
                "y": y1,
            }
        )
        ds_extract_2d = extract_model_to_regular_grid(
            mesh, ds_extract_2d, ["VP", "VS"]
        )
        vpout = ds_extract_2d.VP.values
        vsout = ds_extract_2d.VS.values
        np.save(save_path+"vp{}.npy".format(iS), vpout)
        np.save(save_path+"vs{}.npy".format(iS), vsout)
        np.save(save_path+"src{}.npy".format(iS), srcxy)

        !rm -rf YOUR_PROJECT_DIR

        iS = iS + 1
    
    except ValueError:
        pass
